# Mask R-CNN
Implementation by Matterport



In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import glob
from sklearn import model_selection
from config import Config
import utils
import model as modellib
import visualize
from model import log

# WxH for datasets.
HEIGHT = 280
WIDTH = 320

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to COCO trained weights
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

#Path to dataset, this folder holds train,test and valid datasets. Each folder has two folders (iris,mask)
DATASET_PATH = "/dividedDataset"

## Configurations

In [ ]:
class irisConfig(Config):
    
    
    # Give the configuration a recognizable name
    NAME = "irises"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2
    #MINI_MASK_SHAPE = (84, 84)  
    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 3 shapes
    RPN_ANCHOR_STRIDE = 2
    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 240
    IMAGE_MAX_DIM = 640
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    #Ubiris only
    MEAN_PIXEL = np.array([ 127.8, 90.1, 76.3])
    #IMAGE_SHAPE  = [320,256,3]
    # Use smaller anchors because our image and objects are small
    #RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels
    #MINI_MASK_SHAPE = [40,40]
    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    #TRAIN_ROIS_PER_IMAGE = 48
    
    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 500

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 20    
    #Iris shape, Casia only for now
config = irisConfig()
config.display()



## Notebook Preferences

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Dataset

Create a synthetic dataset

## Must divide iris dataset statically
* load_image()
* load_mask()
* image_reference()

In [ ]:
import re
import skimage
from skimage import io
from sklearn import model_selection
from skimage import util
from numpy import newaxis
import cv2

# Loading the dataset, static folder for now
datasetLoc = ROOT_DIR + DATASET_PATH

numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts
    
        
class irisDataset(utils.Dataset):
    dType = ""
    y_valid = []
    y_train = []
    y_test = []
    def getmaskLoc(self,y_trai,y_vali):
        self.Y_training = y_trai
        self.Y_valid = y_vali
        print (str(len(dataset_train.Y_training)))


    def loadIris(self,dataset):
        print (self.dType)
        os.chdir(ROOT_DIR)
        cwd = os.getcwd()
        print (cwd)
        self.add_class("iris",1,"iris")
        if(dataset == "train"):
            trainIris = datasetLoc + "train/iris"
            os.chdir(trainIris)
            X_train = sorted(glob.glob("*"),key=numericalSort)  

            trainMask = datasetLoc +"train/mask"
            os.chdir(trainMask)
            self.y_train = sorted(glob.glob("*"),key=numericalSort)  
            
            
            for i in range((len(X_train))-1):
                self.add_image("iris",image_id = i,path = datasetLoc + "train/iris/" + X_train[i],width = WIDTH,height = HEIGHT)
        if(dataset == "valid"):

            validIris = datasetLoc + "valid/iris"
            os.chdir(validIris)
            X_valid = sorted(glob.glob("*"),key=numericalSort)  

            validMask = datasetLoc + "valid/mask"
            os.chdir(validMask)
            self.y_valid = sorted(glob.glob("*"),key=numericalSort)  
            
            for i in range((len(X_valid))-1):
                self.add_image("iris",image_id = i,path = datasetLoc + "valid/iris/" +  X_valid[i],width = WIDTH,height = HEIGHT)

        if(dataset == "test"):
            testIris = datasetLoc + "test/iris"
            os.chdir(testIris)
            X_test = sorted(glob.glob("*"),key=numericalSort)  

            testMask = datasetLoc +"test/mask"
            os.chdir(testMask)
            self.y_test = sorted(glob.glob("*"),key=numericalSort)  
            
            for i in range((len(X_test))-1):
                self.add_image("iris",image_id = i,path = datasetLoc + "test/iris/" + X_test[i],width = WIDTH,height = HEIGHT)

        
    def load_mask(self,image_id):
        
        cwd = os.getcwd()
        image = np.ones(shape=(HEIGHT,WIDTH))
        
        if(self.dType == "train"):
            #print (len(self.y_train))

            image = skimage.io.imread(datasetLoc +"train/mask/" +self.y_train[image_id],as_grey=True)
            image = skimage.color.rgb2gray(image)
        if(self.dType == "test"):
            #print (len(self.y_test))

            image = skimage.io.imread(datasetLoc +"test/mask/" +self.y_test[image_id],as_grey=True)
            image = skimage.color.rgb2gray(image)

        if(self.dType == "valid"):
            #print (len(self.y_valid))

            image = skimage.io.imread(datasetLoc +"valid/mask/" +self.y_valid[image_id],as_grey=True)
            image = skimage.color.rgb2gray(image)

        class_ids = np.array([1])
        #New axis due to 1+ number of mask classes in actual implementation
        mask = image[:,:,newaxis]

        for x in range (mask.shape[0]):
            for y in range (mask.shape[1]):
                if mask[x][y] == 1:
                    mask[x][y][0] = 1
        return mask,class_ids
        
os.chdir(ROOT_DIR)

# Training dataset
dataset_train = irisDataset("train")
dataset_train.dType = "train"
dataset_train.loadIris("train")
dataset_train.prepare()

# Validation dataset
dataset_val = irisDataset("valid")
dataset_val.dType = "valid"
dataset_val.loadIris("valid")
dataset_val.prepare()

# Test Dataset
dataset_test = irisDataset("test")
dataset_test.dType = "test"
dataset_test.loadIris("test")
dataset_test.prepare()


In [ ]:
os.chdir(ROOT_DIR + "/logs")
# Displays random samples from each partition of the dataset
image_ids = np.random.choice(dataset_train.image_ids, 2)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    print (image.shape)
    print (image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    print (mask.shape)    
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)
    
image_ids = np.random.choice(dataset_val.image_ids, 2)
for image_id in image_ids:
    image = dataset_val.load_image(image_id)
    print (image_id)
    mask, class_ids = dataset_val.load_mask(image_id)
    print (mask.shape)    
    visualize.display_top_masks(image, mask, class_ids, dataset_val.class_names)
    
image_ids = np.random.choice(dataset_test.image_ids, 2)
for image_id in image_ids:
    image = dataset_test.load_image(image_id)
    print (image_id)
    mask, class_ids = dataset_test.load_mask(image_id)
    print (mask.shape)    
    visualize.display_top_masks(image, mask, class_ids, dataset_test.class_names)

## Model

#### Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [ ]:
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
    print (COCO_MODEL_PATH)
elif init_with == "last":
    model.load_weights(model.find_last()[1], by_name=True)
    print (model.find_last()[1])


## Training

Train in two stages:
1. Training only the head layers

2. Fine tuning left over layers

In [ ]:
import imgaug
from imgaug import augmenters as iaa

              

augmentation = imgaug.augmenters.Fliplr(0.5)

model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=8, 
            layers='all',
            augmentation = augmentation
           )

In [ ]:
augmentation = imgaug.augmenters.Fliplr(0.5)

model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=16, 
            layers='5+',
            augmentation = augmentation
           )

In [ ]:
augmentation = imgaug.augmenters.Fliplr(0.5)

model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=24, 
            layers='all',
            augmentation = augmentation
           )